# Basic RNN
- Objective: to understand basics of RNN & LSTM

## Recurrent Neural Networks
- Feedforward neural networks (e.g. MLPs and CNNs) are powerful, but they are not optimized to handle "sequential" data
- In other words, they do not possess "memory" of previous inputs
- For instance, consider the case of translating a corpus. You need to consider the **"context"** to guess the next word to come forward

<img src="http://2.bp.blogspot.com/-9GIdV292xV4/UwOIy6B6koI/AAAAAAAAHi4/X6UGlyHI-_U/s1600/tumblr_ms5qzpFY671r9nm7io1_500.gif" style="width: 500px"/>

<br>
- RNNs are suitable for dealing with sequential format data since they have **"recurrent"** structure
- To put it differently, they keep the **"memory"** of earlier inputs in the sequence
</br>
<img src="http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg" style="width: 600px"/>

<br>
- However, in order to reduce the number of parameters, every layer of different time steps shares same parameters
</br>

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" style="width: 600px"/>

## Load Dataset

In [1]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

c:\users\heath\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

In [7]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

In [8]:
X_train

array([list([1, 245, 273, 207, 156, 53, 74, 160, 26, 14, 46, 296, 26, 39, 74, 2979, 3554, 14, 46, 4689, 4329, 86, 61, 3499, 4795, 14, 61, 451, 4329, 17, 12]),
       list([1, 486, 341, 785, 26, 14, 482, 26, 255, 606, 252, 83, 146, 91, 102, 17, 12]),
       list([1, 245, 273, 397, 124, 53, 191, 26, 14, 83, 26, 39, 32, 32, 11, 14, 19, 12, 11, 88, 279, 59, 11, 14, 472, 59, 11, 17, 12]),
       ...,
       list([1, 53, 102, 26, 14, 158, 26, 39, 4630, 6694, 14, 3832, 4032, 88, 32, 8204, 5623, 14, 32, 4881, 5006, 29, 53, 841, 26, 14, 425, 26, 39, 5578, 4744, 14, 4693, 2442, 88, 83, 72, 11, 14, 72, 7473, 6173, 17, 12]),
       list([1, 53, 46, 160, 26, 46, 5352, 2637, 86, 160, 5191, 3302, 123, 943, 396, 4500, 207, 48, 17, 12]),
       list([1, 3840, 341, 353, 26, 14, 61, 19, 26, 255, 219, 93, 124, 146, 93, 102, 123, 4433, 274, 1146, 131, 664, 338, 721, 17, 12])],
      dtype=object)

In [9]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

In [10]:
X_train

array([[   1,  245,  273, ...,    0,    0,    0],
       [   1,  486,  341, ...,    0,    0,    0],
       [   1,  245,  273, ...,    0,    0,    0],
       ...,
       [   1,   53,  102, ...,    0,    0,    0],
       [   1,   53,   46, ...,    0,    0,    0],
       [   1, 3840,  341, ...,    0,    0,    0]])

In [11]:
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

In [12]:
y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

In [13]:
y_train = y_data[:1395]
y_test = y_data[1395:]

In [14]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1395, 49, 1)
(599, 49, 1)
(1395, 46)
(599, 46)


## 1. Vanilla RNN
- Vanilla RNNs have a simple structure
- However, they suffer from the problem of "long-term dependencies"
- Hence, they are not able to keep the **sequential memory" for long

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" style="width: 600px"/>

In [20]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

In [21]:
def vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [22]:
model = KerasClassifier(build_fn = vanilla_rnn, epochs = 200, batch_size = 50, verbose = 1)

In [23]:
model.fit(X_train, y_train)

Epoch 1/200
1395/1395 [==============================] - 1s 634us/step - loss: 2.9124 - acc: 0.5011
Epoch 2/200
1395/1395 [==============================] - 1s 415us/step - loss: 1.3720 - acc: 0.7082
Epoch 3/200
1395/1395 [==============================] - 1s 397us/step - loss: 1.1989 - acc: 0.7147
Epoch 4/200
1395/1395 [==============================] - 1s 385us/step - loss: 1.1721 - acc: 0.7147
Epoch 5/200
1395/1395 [==============================] - 1s 417us/step - loss: 1.1641 - acc: 0.7147
Epoch 6/200
1395/1395 [==============================] - 1s 441us/step - loss: 1.1608 - acc: 0.7147
Epoch 7/200
1395/1395 [==============================] - 1s 426us/step - loss: 1.1572 - acc: 0.7147
Epoch 8/200
1395/1395 [==============================] - 1s 434us/step - loss: 1.1576 - acc: 0.7147
Epoch 9/200
1395/1395 [==============================] - 1s 437us/step - loss: 1.1547 - acc: 0.7147
Epoch 10/200
1395/1395 [==============================] - 1s 481us/step - loss: 1.1531 - acc: 0.7147

1395/1395 [==============================] - 1s 446us/step - loss: 0.9744 - acc: 0.7283
Epoch 82/200
1395/1395 [==============================] - 1s 414us/step - loss: 0.9672 - acc: 0.7276
Epoch 83/200
1395/1395 [==============================] - 1s 452us/step - loss: 0.9592 - acc: 0.7262
Epoch 84/200
1395/1395 [==============================] - 1s 431us/step - loss: 0.9660 - acc: 0.7219
Epoch 85/200
1395/1395 [==============================] - 1s 404us/step - loss: 0.9601 - acc: 0.7276 0s - loss: 0.9560 - acc: 0
Epoch 86/200
1395/1395 [==============================] - 1s 400us/step - loss: 0.9513 - acc: 0.7355
Epoch 87/200
1395/1395 [==============================] - 1s 472us/step - loss: 0.9459 - acc: 0.7348
Epoch 88/200
1395/1395 [==============================] - 1s 430us/step - loss: 0.9480 - acc: 0.7312
Epoch 89/200
1395/1395 [==============================] - 1s 412us/step - loss: 0.9505 - acc: 0.7341
Epoch 90/200
1395/1395 [==============================] - 1s 466us/step - los

1395/1395 [==============================] - 1s 413us/step - loss: 0.7890 - acc: 0.7548
Epoch 162/200
1395/1395 [==============================] - 1s 406us/step - loss: 0.7872 - acc: 0.7599
Epoch 163/200
1395/1395 [==============================] - 1s 414us/step - loss: 0.7865 - acc: 0.7577
Epoch 164/200
1395/1395 [==============================] - 1s 396us/step - loss: 0.7894 - acc: 0.7577
Epoch 165/200
1395/1395 [==============================] - 1s 401us/step - loss: 0.7856 - acc: 0.7584
Epoch 166/200
1395/1395 [==============================] - 1s 406us/step - loss: 0.7804 - acc: 0.7563
Epoch 167/200
1395/1395 [==============================] - 1s 427us/step - loss: 0.7798 - acc: 0.7577
Epoch 168/200
1395/1395 [==============================] - 1s 399us/step - loss: 0.7740 - acc: 0.7570
Epoch 169/200
1395/1395 [==============================] - 1s 418us/step - loss: 0.7739 - acc: 0.7591
Epoch 170/200
1395/1395 [==============================] - 1s 387us/step - loss: 0.7789 - acc: 0

In [24]:
y_pred = model.predict(X_test)

599/599 [==============================] - 0s 253us/step


In [25]:
y_test_ = np.argmax(y_test, axis = 1)

In [70]:
print(accuracy_score(y_pred, y_test_))

0.74958263773


## 2. Stacked Vanilla RNN
- RNN layers can be stacked to form a deeper network

<img src="https://lh6.googleusercontent.com/rC1DSgjlmobtRxMPFi14hkMdDqSkEkuOX7EW_QrLFSymjasIM95Za2Wf-VwSC1Tq1sjJlOPLJ92q7PTKJh2hjBoXQawM6MQC27east67GFDklTalljlt0cFLZnPMdhp8erzO" style="width: 500px"/>

In [74]:
def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(50, input_shape = (49,1), return_sequences = True))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(50, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [75]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 200, batch_size = 50, verbose = 1)

In [ ]:
model.fit(X_train, y_train)

In [77]:
y_pred = model.predict(X_test)

500/599 [========================>.....] - ETA: 0s

In [78]:
print(accuracy_score(y_pred, y_test_))

0.746243739566


## 3. LSTM
- LSTM (long short-term memory) is an improved structure to solve the problem of long-term dependencies

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" style="width: 600px"/>

In [15]:
from keras.layers import LSTM

In [17]:
def lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [18]:
model = KerasClassifier(build_fn = lstm, epochs = 200, batch_size = 50, verbose = 1)

In [19]:
model.fit(X_train, y_train)

Epoch 1/200
1395/1395 [==============================] - 5s 3ms/step - loss: 3.4349 - acc: 0.5527
Epoch 2/200
1395/1395 [==============================] - 2s 2ms/step - loss: 1.5083 - acc: 0.7147
Epoch 3/200
1395/1395 [==============================] - 2s 2ms/step - loss: 1.1920 - acc: 0.7147
Epoch 4/200
1000/1395 [====================>.........] - ETA: 0s - loss: 1.1524 - acc: 0.7140

KeyboardInterrupt: 

In [83]:
y_pred = model.predict(X_test)

500/599 [========================>.....] - ETA: 0s

In [84]:
# accuracy improves by adopting LSTM structure
print(accuracy_score(y_pred, y_test_))

0.844741235392


## 4. Stacked LSTM
- LSTM layers can be stacked as well

In [85]:
def stacked_lstm():
    model = Sequential()
    model.add(LSTM(50, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(50, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [86]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 200, batch_size = 50, verbose = 1)

In [ ]:
model.fit(X_train, y_train)

In [88]:
y_pred = model.predict(X_test)

500/599 [========================>.....] - ETA: 0s

In [89]:
print(accuracy_score(y_pred, y_test_))

0.858096828047
